In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [6]:
df=pd.read_csv('train_storming_round.csv')
df['agent_join_month']=pd.to_datetime(df['agent_join_month'])
df['year_month']=pd.to_datetime(df['year_month'])
df['first_policy_sold_month']=pd.to_datetime(df['first_policy_sold_month'])
df

,row_id,agent_code,agent_age,agent_join_month,first_policy_sold_month,year_month,unique_proposals_last_7_days,unique_proposals_last_15_days,unique_proposals_last_21_days,unique_proposal,...,unique_quotations,unique_customers_last_7_days,unique_customers_last_15_days,unique_customers_last_21_days,unique_customers,new_policy_count,ANBP_value,net_income,number_of_policy_holders,number_of_cash_payment_policies
0,1,455ca878,45,2021-02-01,2023-09-01,2023-01-01,3,6,2,12,...,9,3,4,8,15,0,0,93313,30,162
1,2,c823ce77,48,2022-04-01,2024-02-01,2023-01-01,1,4,12,21,...,14,4,3,10,17,25,1071450,164804,0,175
2,3,62154055,53,2020-05-01,2023-09-01,2023-01-01,3,5,5,17,...,14,3,7,11,21,26,1601210,426690,84,78
3,4,c58bfa6e,44,2019-07-01,2022-03-01,2023-01-01,1,0,12,17,...,11,2,2,9,13,18,283230,47793,39,144
4,5,b1e5f770,20,2020-09-01,2023-02-01,2023-01-01,0,6,10,17,...,13,3,10,6,19,20,1957680,562239,19,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15303,15304,d3b44e44,43,2019-04-01,2023-10-01,2024-08-01,1,6,6,17,...,10,3,4,11,18,25,2473900,404980,18,100
15304,15305,0b1d7054,28,2022-09-01,2024-03-01,2024-08-01,1,5,3,13,...,14,4,3,11,18,15,922050,198187,18,120
15305,15306,51321fa2,45,2021-01-01,2024-04-01,2024-08-01,0,6,17,26,...,22,6,7,6,19,30,645330,133263,57,270
15306,15307,dd4a0d2d,25,2021-12-01,2023-12-01,2024-08-01,0,4,6,14,...,13,6,3,6,15,0,0,32546,30,171


In [10]:
# Group agent-level performance
agent_perf = df.groupby('agent_code').agg({'new_policy_count': ['sum', 'mean']})
agent_perf.columns = ['total_sales', 'avg_sales_per_month']
agent_perf = agent_perf.reset_index()

In [13]:
agent_perf

,agent_code,total_sales,avg_sales_per_month
0,003c1999,448,22.40
1,00d5bda3,405,20.25
2,012a3553,362,18.10
3,012f0e15,393,19.65
4,0177ed5f,425,21.25
...,...,...,...
900,ff8edcc9,428,21.40
901,ff955d18,26,13.00
902,ffca3177,413,20.65
903,ffe6c9e9,391,19.55


In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(agent_perf[['total_sales', 'avg_sales_per_month']])

In [17]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
agent_perf['cluster'] = kmeans.fit_predict(X_scaled)

In [25]:
cluster_summary = agent_perf.groupby('cluster')['total_sales'].mean().sort_values()

In [ ]:
cluster_summary

cluster
1    126.710280
2    253.356784
0    411.213689
Name: total_sales, dtype: float64

In [34]:
performance_labels = ['Low', 'Medium', 'High']

# Step 2: Map cluster IDs to labels based on sorted order
cluster_label_map = {cluster_id: performance_labels[i] for i, cluster_id in enumerate(cluster_summary.index)}

# Step 3: Apply mapping to your agent_perf DataFrame
agent_perf['performance'] = agent_perf['cluster'].map(cluster_label_map)

In [ ]:
agent_perf

,agent_code,total_sales,avg_sales_per_month,cluster,performance
0,003c1999,448,22.40,0,High
1,00d5bda3,405,20.25,0,High
2,012a3553,362,18.10,2,Medium
3,012f0e15,393,19.65,0,High
4,0177ed5f,425,21.25,0,High
...,...,...,...,...,...
900,ff8edcc9,428,21.40,0,High
901,ff955d18,26,13.00,2,Medium
902,ffca3177,413,20.65,0,High
903,ffe6c9e9,391,19.55,0,High


In [37]:
selected_columns = ['agent_code', 'performance']  # replace with your actual columns
df_selected = agent_perf[selected_columns]

# Save to CSV
df_selected.to_csv('selected_output.csv', index=False)